# 1. Import Data



In [1]:
#Import Dependencies 
from nilearn import plotting
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from nilearn import image
import nibabel as nib
import os
from nilearn.connectome import ConnectivityMeasure
from nilearn.input_data import NiftiLabelsMasker
from brainspace.plotting import plot_hemispheres
from brainspace.datasets import load_parcellation, load_conte69
from brainspace.gradient import GradientMaps
labeling = load_parcellation('schaefer', scale=100, join=True)
# and load the conte69 surfaces
surf_lh, surf_rh = load_conte69()

/Users/mioulin/opt/anaconda3/lib/python3.9/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
# Load Schefer atlas for parcellation # 100 ROI, 7Networks
from nilearn import plotting, datasets
atlas_file = datasets.fetch_atlas_schaefer_2018(n_rois=100, yeo_networks=7,resolution_mm=2)
atlas_filename = atlas_file.maps
labels = atlas_file.labels

In [3]:
# Import Masker 
from nilearn.input_data import NiftiLabelsMasker
masker = NiftiLabelsMasker(labels_img=atlas_filename, standardize=True,
                           memory='nilearn_cache', verbose=5)
masker

NiftiLabelsMasker(labels_img='/Users/mioulin/nilearn_data/schaefer_2018/Schaefer2018_100Parcels_7Networks_order_FSLMNI152_2mm.nii.gz',
                  memory='nilearn_cache', standardize=True, verbose=5)

In [4]:
atlas_file = nib.load('/Users/mioulin/nilearn_data/schaefer_2018/Schaefer2018_100Parcels_7Networks_order_FSLMNI152_2mm.nii.gz')

with open('/Users/mioulin/nilearn_data/schaefer_2018/Schaefer2018_100Parcels_7Networks_order.txt', 'r') as f:
    labels = f.read().split('\n')
masker = NiftiLabelsMasker(labels_img=atlas_file, standardize=True, verbose=1,
                           memory="nilearn_cache", memory_level=2)
labels = [x.split('\t')[1][10:] for x in labels]


# 2. LSD Timeseries extraction

In [4]:
import os
data_path =  '/Users/mioulin/Desktop/data/lsd/lsd_scans'
subjects = ['001', '002', '004', '006', '009', '010', '011', '013', '017', '018', '019', '020']

sessions = ['drug','pcb']

In [150]:
task = 'rest'
runs = ['01','02','03']
suffix = 'nii.gz'

timeseries_all = dict()

#Iterationg over sessions
for ses in sessions:
    
    #Iterating over subjects
    for i, subj in enumerate(subjects):
        
        #Iterating over runs
        for k, run in enumerate(runs):
            file_path = os.path.join(data_path, f'sub-{subj}', f'ses-{ses}', 'func', f'sub-{subj}_ses-{ses}_task-{task}_run-{run}_bold.{suffix}')
            timeseries_all[run,ses,subj] =  masker.fit_transform(file_path, confounds=None)

[NiftiLabelsMasker.fit_transform] loading data from /Users/mioulin/nilearn_data/schaefer_2018/Schaefer2018_100Parcels_7Networks_order_FSLMNI152_2mm.nii.gz
[Memory]9942.9s, 165.7min: Loading filter_and_extract...
__________________________________filter_and_extract cache loaded - 0.0s, 0.0min
[NiftiLabelsMasker.fit_transform] loading data from /Users/mioulin/nilearn_data/schaefer_2018/Schaefer2018_100Parcels_7Networks_order_FSLMNI152_2mm.nii.gz
[Memory]9945.2s, 165.8min: Loading filter_and_extract...
__________________________________filter_and_extract cache loaded - 0.0s, 0.0min
[NiftiLabelsMasker.fit_transform] loading data from /Users/mioulin/nilearn_data/schaefer_2018/Schaefer2018_100Parcels_7Networks_order_FSLMNI152_2mm.nii.gz
[Memory]9947.9s, 165.8min: Loading filter_and_extract...
__________________________________filter_and_extract cache loaded - 0.0s, 0.0min
[NiftiLabelsMasker.fit_transform] loading data from /Users/mioulin/nilearn_data/schaefer_2018/Schaefer2018_100Parcels_7Ne

In [153]:
import pickle

# Save the dictionary as a binary file using pickle
with open('timeseries_all.pkl', 'wb') as f:
    pickle.dump(timeseries_all, f)

In [8]:
with open('timeseries_all.pkl', 'rb') as f:
    back = pickle.load(f)
    
#back


In [152]:
timeseries_all


{('01',
  'LSD',
  '001'): array([[ 0.55105025,  0.5591349 , -0.38001975, ...,  0.71009785,
          0.3071096 ,  0.73192096],
        [ 0.5633694 ,  0.6090556 , -0.37742892, ...,  0.7391166 ,
          0.39906853,  0.80617875],
        [ 0.57568854,  0.81089103, -0.19144408, ...,  1.1627618 ,
          0.5838746 ,  0.8677301 ],
        ...,
        [-0.9159027 ,  1.1626049 ,  0.48804826, ...,  1.0458249 ,
          2.1202736 ,  1.8447633 ],
        [-0.44321722,  1.0990695 , -0.1546635 , ...,  1.0143639 ,
          2.2910914 ,  2.0159593 ],
        [ 0.15955168,  0.58060426, -0.59755725, ...,  0.9485689 ,
          1.7926999 ,  1.7858763 ]], dtype=float32),
 ('02',
  'LSD',
  '001'): array([[ 0.06775026,  0.63120687,  0.4121453 , ...,  0.02160084,
         -0.8816158 , -0.6491314 ],
        [ 0.05694499,  0.6778174 ,  0.4320096 , ...,  0.08438911,
         -0.8345853 , -0.6161771 ],
        [ 0.8837962 ,  0.87774086,  0.7067907 , ..., -0.20475647,
         -0.7696089 , -0.60029846],


In [154]:
ts_lsd = {}

# Iterate over the dictionary items and separate the keys
for key, value in timeseries_all.items():
    key1, key2, key3 = key
    if key1 not in ts_lsd:
        ts_lsd[key1] = {}
    if key2 not in ts_lsd[key1]:
        ts_lsd[key1][key2] = {}
    ts_lsd[key1][key2][key3] = value

# Print the nested dictionary
ts_lsd

{'01': {'LSD': {'001': array([[ 0.55105025,  0.5591349 , -0.38001975, ...,  0.71009785,
            0.3071096 ,  0.73192096],
          [ 0.5633694 ,  0.6090556 , -0.37742892, ...,  0.7391166 ,
            0.39906853,  0.80617875],
          [ 0.57568854,  0.81089103, -0.19144408, ...,  1.1627618 ,
            0.5838746 ,  0.8677301 ],
          ...,
          [-0.9159027 ,  1.1626049 ,  0.48804826, ...,  1.0458249 ,
            2.1202736 ,  1.8447633 ],
          [-0.44321722,  1.0990695 , -0.1546635 , ...,  1.0143639 ,
            2.2910914 ,  2.0159593 ],
          [ 0.15955168,  0.58060426, -0.59755725, ...,  0.9485689 ,
            1.7926999 ,  1.7858763 ]], dtype=float32),
   '002': array([[-1.5423176 ,  0.41512075, -0.303112  , ..., -1.0999124 ,
           -0.5420789 ,  0.6607987 ],
          [-1.6065009 ,  0.41082525, -0.34717456, ..., -1.1202035 ,
           -0.56779146,  0.6579216 ],
          [-1.0433027 ,  0.90366787, -0.4411247 , ..., -0.75808436,
           -1.3342313 ,  

# 3. Import Gradient 

In [147]:
# Load hcp functional connectivitty from brainspace toolbox: 
from brainspace.datasets import load_group_fc, load_parcellation, load_conte69

# First load mean connectivity matrix and Schaefer parcellation
conn_matrix = load_group_fc('schaefer', scale=100)
labeling = load_parcellation('schaefer', scale=100, join=True)
from brainspace.gradient import GradientMaps

# Ask for one fc hcp grad
gm = GradientMaps(n_components=3, random_state=0)
gm.fit(conn_matrix)
grad_hcp = gm.gradients_


/Users/mioulin/opt/anaconda3/lib/python3.9/site-packages/brainspace/gradient/embedding.py:70: UserWarning: Affinity is not symmetric. Making symmetric.
  warnings.warn('Affinity is not symmetric. Making symmetric.')


In [172]:
def vcorrelate(X,y):
    
    #Create input for vector
    
    N = X.shape[0]
    k = y.shape[0]
    r = np.zeros((N,k))
    
    # output
    for j in range(k):
        for i in range(N):
            r[i,j] = np.corrcoef(X[i,:], y[j])[0,1]
    return r

In [178]:
import numpy as np

corr_results_lsd = {}
b = grad_hcp.T

for rest, states in ts_lsd.items():
    corr_results_lsd[rest] = {}
    for state, subjects in states.items():
        corr_results_lsd[rest][state] = {}
        for subject, series in subjects.items():
            X = np.array(series)  # Convert the series data to a NumPy array
            corr_results_lsd[rest][state][subject] = vcorrelate(X, b)

In [180]:
corr_rest1 = corr_results_lsd['01']
corr_rest2 = corr_results_lsd['02']
corr_rest3 = corr_results_lsd['03']

In [181]:
import pickle

# Save the dictionary as a binary file using pickle
with open('corr_rest1.pkl', 'wb') as f:
    pickle.dump(corr_rest1, f)
with open('corr_rest2.pkl', 'wb') as f:
    pickle.dump(corr_rest2, f)
with open('corr_rest3.pkl', 'wb') as f:
    pickle.dump(corr_rest3, f)
